In [4]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os 

In [5]:
train_path = 'G:\\PYTHON\\Final year project\\Dataset\\train'
val_path = 'G:\\PYTHON\\Final year project\\Dataset\\val'
test_path = 'G:\\PYTHON\\Final year project\\Dataset\\test'
#test_path= test_data

In [6]:
# re-size all the images to a size VGG-16 expects.
IMAGE_SIZE = [224, 224]

# Set the batch size
BATCH_SIZE = 16  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001
NUM_CLASSES = 2 # We are aware of it.

In [7]:
CLASSES = os.listdir(train_path)
NUM_CLASSES = len(CLASSES)
print("Class --> {} \n and the length is : {}".format(CLASSES, NUM_CLASSES))

Class --> ['covid', 'normal'] 
 and the length is : 2


In [8]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [9]:
# Import the images from the train dataset.
# Make sure to provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 6030 images belonging to 2 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
# Import the images from the test dataset.

test_set = test_datagen.flow_from_directory(
    directory = test_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 860 images belonging to 2 classes.


In [12]:
resnet = ResNet50(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94781440/94765736 [==============================] - 8s 0us/step


In [13]:
for layer in resnet.layers:
    layer.trainable = False

In [14]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

prediction = Dense(NUM_CLASSES, activation='softmax')(x)

In [15]:
model = Model(inputs=resnet.input, outputs=prediction)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [17]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
%%timeit -n1 -r1 
with tf.device('/GPU:0'):
    history = model.fit(
        training_set,
        validation_data=test_set,
        epochs=8,
        steps_per_epoch=len(training_set),
        validation_steps=len(test_set)
    )

Epoch 1/8
377/377 [==============================] - 89s 236ms/step - loss: 0.5972 - accuracy: 0.7889 - val_loss: 0.6886 - val_accuracy: 0.6872
Epoch 2/8
377/377 [==============================] - 112s 297ms/step - loss: 0.5612 - accuracy: 0.7982 - val_loss: 0.6694 - val_accuracy: 0.6849
Epoch 3/8
377/377 [==============================] - 132s 350ms/step - loss: 0.5270 - accuracy: 0.8022 - val_loss: 0.8287 - val_accuracy: 0.6814
Epoch 4/8
377/377 [==============================] - 120s 317ms/step - loss: 0.5463 - accuracy: 0.8012 - val_loss: 1.1515 - val_accuracy: 0.6419
Epoch 5/8
377/377 [==============================] - 136s 362ms/step - loss: 0.4746 - accuracy: 0.8151 - val_loss: 0.6399 - val_accuracy: 0.7093
Epoch 6/8
377/377 [==============================] - 156s 413ms/step - loss: 0.5181 - accuracy: 0.8056 - val_loss: 0.6708 - val_accuracy: 0.7000
Epoch 7/8
377/377 [==============================] - 163s 433ms/step - loss: 0.8402 - accuracy: 0.7715 - val_loss: 1.1154 - val_acc

In [22]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 1726 images belonging to 2 classes.


In [23]:
validation_steps = 200

loss0,accuracy0 = model.evaluate(validation_set, steps = validation_steps)

print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

200/200 [==============================] - 14s 70ms/step - loss: 0.7901 - accuracy: 0.7231
loss: 0.79
accuracy: 0.72


In [41]:
# Generate Validation set.
validation_set2 = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = 1,
    shuffle=False, 
    seed=42, 
    class_mode="binary"
)

# validation_set2.reset()

Found 1726 images belonging to 2 classes.


In [42]:
# just capture the loss and accuray into val variable... unlike in pervious code to capture into loss0 and accuracy0. Just to showcase alternate way.

test = model.evaluate(test_set)

print("loss: {:.2f}".format(test[0]))
print("accuracy: {:.2f}".format(test[1]))

54/54 [==============================] - 6s 106ms/step - loss: 0.9183 - accuracy: 0.6791
loss: 0.92
accuracy: 0.68


In [43]:
model.save("G:\\PYTHON\\Final year project\\weights\\resnet_50_weights.h5")

C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
